In [1]:
# 필요할 라이브러리 불러오기
import tensorflow as tf
import keras
import numpy as np
from keras.preprocessing import image

# 전이학습 모델
from tensorflow.keras.applications import EfficientNetV2B1

# 예측
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential, load_model

In [2]:
# Model 불러오기
EfficientNet_V2B1_model = load_model('/content/drive/MyDrive/CP1 데이터/EfficientNet_V2B1_model.h5')

In [3]:
def class_predic():
    img = keras.preprocessing.image.load_img(f"{input()}", target_size=(256,256))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis

    predictions = EfficientNet_V2B1_model.predict(img_array)
    score = predictions[0]

    class_names = ('nevus(모반)', 'malignant(흑색종 악성)', 'benign(흑색종 양성)', 'seborrheic_keratosis(지루성 각화증)', 'BCC(기저 세포 암종)')

    for i in range(len(class_names)):
        if np.argmax(score) == i:
            print(class_names[i])

In [4]:
class_predic()

/content/drive/MyDrive/CP1 데이터/nevus(모반)/ISIC_0000093.jpg
nevus(모반)
